## Project 1: Robot Path Planning 

In [16]:
# Import libaries
import math
import heapq
import altair as alt
import pandas as pd

In [17]:
# Read in input file store start coordinate, end coordinate, and the grid matrix
def read_input_file(file_path):
    with open(file_path, 'r') as file:
        first_line = file.readline().strip()
        start_x, start_y, goal_x, goal_y = map(int, first_line.split())
        grid_matrix = []
        for x in range(30):  
            line = file.readline().strip()
            if line: 
                cell_values = list(map(int, line.split()))
                grid_matrix.append([[x, y, cell_values[y]] for y in range(len(cell_values))])

    return (start_x, start_y), (goal_x, goal_y), grid_matrix

start_coord, goal_coord, grid_matrix = read_input_file("input1.txt")
print(start_coord)
print(goal_coord)
print(grid_matrix)

(1, 6)
(27, 27)
[[[0, 0, 0], [0, 1, 0], [0, 2, 0], [0, 3, 0], [0, 4, 0], [0, 5, 0], [0, 6, 0], [0, 7, 0], [0, 8, 0], [0, 9, 0], [0, 10, 0], [0, 11, 1], [0, 12, 0], [0, 13, 0], [0, 14, 0], [0, 15, 0], [0, 16, 0], [0, 17, 0], [0, 18, 1], [0, 19, 0], [0, 20, 0], [0, 21, 0], [0, 22, 0], [0, 23, 0], [0, 24, 0], [0, 25, 0], [0, 26, 0], [0, 27, 0], [0, 28, 1], [0, 29, 0], [0, 30, 0], [0, 31, 0], [0, 32, 0], [0, 33, 0], [0, 34, 0], [0, 35, 0], [0, 36, 0], [0, 37, 0], [0, 38, 1], [0, 39, 1], [0, 40, 0], [0, 41, 0], [0, 42, 0], [0, 43, 0], [0, 44, 0], [0, 45, 0], [0, 46, 0], [0, 47, 0], [0, 48, 1], [0, 49, 0]], [[1, 0, 0], [1, 1, 0], [1, 2, 0], [1, 3, 0], [1, 4, 0], [1, 5, 0], [1, 6, 2], [1, 7, 1], [1, 8, 0], [1, 9, 0], [1, 10, 0], [1, 11, 0], [1, 12, 1], [1, 13, 0], [1, 14, 0], [1, 15, 0], [1, 16, 0], [1, 17, 0], [1, 18, 0], [1, 19, 0], [1, 20, 0], [1, 21, 1], [1, 22, 0], [1, 23, 0], [1, 24, 0], [1, 25, 0], [1, 26, 0], [1, 27, 1], [1, 28, 0], [1, 29, 0], [1, 30, 0], [1, 31, 0], [1, 32, 0], [1, 

In [18]:
# Visualize the robot workspace
flat_grid = [cell for row in grid_matrix for cell in row]
df = pd.DataFrame(flat_grid, columns=['x', 'y', 'cell_value'])
color_scale = alt.Scale(domain=[0, 1, 2, 5],
                        range=['white', 'black', 'red', 'green'])

heatmap = alt.Chart(df).mark_square(size=220).encode(
    x=alt.X('y:O', axis=alt.Axis(labels=False, title=None)),  
    y=alt.Y('x:O', axis=alt.Axis(labels=False, title=None), sort='descending'), 
    color=alt.Color('cell_value:N', scale=color_scale)  
).properties(
    width=alt.Step(10), 
    height=alt.Step(10)
)

horizontal_grid = alt.Chart(pd.DataFrame({'y': [i + 0.5 for i in range(30)] * 30})).mark_rule(color='gray', strokeWidth=0.5).encode(
    y='y:O'
)

vertical_grid = alt.Chart(pd.DataFrame({'x': [i + 0.5 for i in range(50)] * 50})).mark_rule(color='gray', strokeWidth=0.5).encode(
    x='x:O'
)

heatmap = heatmap + horizontal_grid + vertical_grid
heatmap

alt.LayerChart(...)

In [19]:
class Node:
    def __init__(self, state, parent=None, action=None, g=0, h=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.g = g  
        self.h = h  
    
    def f(self):
        return self.g + self.h
    
    def __lt__(self, other):
        return self.f() < other.f()

actions = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)]

def is_valid_move(x, y, grid_matrix):
    return 0 <= x < len(grid_matrix) and 0 <= y < len(grid_matrix[0]) and grid_matrix[x][y] == 0

def euclidean_distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def a_star_search(start_coord, goal_coord, grid_matrix):
    open_list = []
    closed_set = set()
    start_node = Node(start_coord, None, None, 0, euclidean_distance(start_coord, goal_coord))
    heapq.heappush(open_list, start_node)
    
    while open_list:
        current_node = heapq.heappop(open_list)
        closed_set.add(current_node.state)
        
        if current_node.state == goal_coord:
            return current_node
        
        for action in actions:
            new_x = current_node.state[0] + action[0]
            new_y = current_node.state[1] + action[1]
            if is_valid_move(new_x, new_y, grid_matrix) and (new_x, new_y) not in closed_set:
                new_g = current_node.g + (1 if action[0] == 0 or action[1] == 0 else math.sqrt(2))
                new_h = euclidean_distance((new_x, new_y), goal_coord)
                new_node = Node((new_x, new_y), current_node, action, new_g, new_h)
                heapq.heappush(open_list, new_node)
    
    return None

def extract_path(goal_node):
    path = []
    f_values = []
    current_node = goal_node
    while current_node is not None:
        path.append(current_node.action)
        f_values.append(current_node.f())
        current_node = current_node.parent
    path.reverse()
    f_values.reverse()
    return path, f_values

def write_output_file(goal_node, grid_matrix, output_file):
    path, f_values = extract_path(goal_node)
    depth_level = len(path) - 1
    total_nodes = len(path) + 1
    
    with open(output_file, 'w') as file:
        file.write(str(depth_level) + '\n')
        file.write(str(total_nodes) + '\n')
        file.write(' '.join(str(action) for action in path) + '\n')
        file.write(' '.join(str(f) for f in f_values) + '\n')
        for row in grid_matrix:
            file.write(' '.join(str(cell[2]) for cell in row) + '\n')

In [20]:
goal_node = a_star_search(start_coord, goal_coord, grid_matrix)
if goal_node:
    write_output_file(goal_node, grid_matrix, "output.txt")
    print("Output written to output.txt")
else:
    print("No path found")

No path found
